In [15]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import ast
from experimental_metrics import *

In [16]:
popularity_df = pd.read_csv("dataset/id_metadata_mmsr.tsv", sep="\t")
popularity = popularity_df['popularity'].values
pop = pd.read_csv("dataset/id_total_listens.tsv", sep="\t")['total_listens'].values
infos = pd.read_csv("dataset/id_information_mmsr.tsv", sep="\t")
inter_true = np.loadtxt("./predictions/binary_relevancy_matrix_00.csv", delimiter="\t")
tags = pd.read_csv("./dataset/id_tags_dict.tsv", sep="\t")
genres = pd.read_csv("./dataset/id_genres_mmsr.tsv", sep="\t")

In [17]:
models = [  
    # Baseline
    #'random',
    
    # TEXT
    #"lyrics_tf-idf", 
    #"lyrics_bert",
    #"lyrics_word2vec",

    # AUDIO
    #"ivec256", 
    #"mfcc_bow",
    #"mfcc_stats",
    #"musicnn",

    # VIDEO
    #"resnet",
    #"vgg19", 
    #"incp",

    # Hybrid
    'early_fusion',
    #'late_fusion',
    #'quick_late_fusion',

    # Creative
    'gm_nn',
    #'gm_comm',
    #'mgm_nn',
    #'mgm_comm',

    # Diversity Aware
    #'shifted',
    #'p_shifted',
    #'exp',
    #'power',
    #'mmr'
]
evaluation = []
topK=100

for model in models:
    inter_pred = np.loadtxt(f"predictions/rets_{model}_{topK}_matrix.csv", delimiter=",")
    pop_at_k = avg_popularity_at_k(inter_pred, popularity, topK)
    pop_at_k_2 = avg_popularity_at_k(inter_pred, pop, topK)
    cov_at_k = avg_coverage_at_k(inter_pred, inter_true.shape[0], topK)
    div_at_k = diversity_at_k(inter_pred,tags, genres, k=topK)

    evaluation.append({
        "model": model,
        f"pop-sp@{topK}":    pop_at_k,
        f"pop-lfm@{topK}":   pop_at_k_2,
        f"cov@{topK}":       cov_at_k,
        f"div@{topK}":       div_at_k
    })

In [ ]:
evaluation_df = pd.DataFrame(evaluation)